We would like to minimize following optimization using splitting.
$$min_{H \in R^{n \times p}} Tr(H^TH)$$
$$s.t. H^T H  = I$$

The SOC method works as such. <img src='main.png'>

After splitting so, the first subproblem is a convex optimization problem, then easy to solve. Second one will be solved with respect to following remark mentioned in the paper. <img src='alg1.png'>

So, these are the steps need to be done. <br />
<img src='alg2.png'>

It is obvious the optimal solution of our toy problem is m, the number of columns of H. Let's implement this algorithm and see how it does. I  our problem $L$ is $I$ and the subproblems are:
$$ H^k = argmin_{H} Tr(H^TH)+\frac{r}{2}\|H-P^{k-1}+B^{k-1}\|_{F}^2 $$
$$ P^k = argmin_{p} \frac{r}{2} \|P-(H^k+B^{k-1})\|_{F}^2  \quad s.t \quad   P^TP=I$$


The first problem has an analytical solution, easily computed by taking derivative as follows.
$$2H+\frac{r}{2}(2H-2(P^{k-1}-B^{k-1}))$$
$$H = \frac{r}{(2+r)} (P^{k-1}-B^{k-1})$$

In [1]:
#
import numpy as np
from scipy.optimize import minimize
%matplotlib inline

In [50]:
iter = 1000
dimension = 10 # assume H is 3*3 so the optimal value should be 3

#initialization based on the sketch of algorithm
H =np.arange(dimension**2).reshape((dimension,dimension)) #This is just initial value of H
H = np.random.randn(dimension,2)
P =H 
B =np.zeros(shape=(dimension,2))
r=300
L  = np.random.randn(dimension,dimension)
L =0.5*( L.T+L)

for i in range(iter):
#     H = (P-B)*(r/(2.0+r))
    H = np.linalg.solve(2*L+r*np.identity(dimension),r*(P-B))
    y_k = H+B
    u,d,v = np.linalg.svd(y_k,full_matrices=True)
    
    P=u.dot(np.eye(N=u.shape[1],M=v.shape[0])).dot(v)
    if  not np.allclose(np.dot(P.T,P), np.identity(n=P.T.shape[0])):
        print "Constrained subproblem failed"
        break

    B = B+H-P

print "Is an orthogonal constraint statisifed? Is the printed matrix equal to I?",'\n'
print np.round(np.linalg.norm(H.T.dot(H)-np.identity(H.shape[1])))
print "objective is ",np.trace(H.T.dot(H)) ,np.allclose(np.trace(H.T.dot(H)),H.T.shape[0])
from scipy.linalg import eigh
lambads,vec = eigh(L,eigvals=(0,1))
# vec = vec[:,:2]
print np.linalg.norm(vec-H)
print np.trace(vec.T.dot(L).dot(vec)),np.trace(H.T.dot(L).dot(H))

Is an orthogonal constraint statisifed? Is the printed matrix equal to I? 

0.0
objective is  2.00000000535 True
1.44377454883
-7.18757149748 -7.18754027322


In [48]:

l,emb = eigh(L,eigvals=(0,1))
print np.linalg.norm(emb-vec)
print np.trace(emb.T.dot(L).dot(emb))

2.0
-26.3345617131


we also can solve the first problem numerically by the tools in the scipy.optimize library.

In [3]:
def objective(H,P,B):
    
    H=H.reshape(dimension,dimension)
    y=B-P
    return np.trace(H.T.dot(H))+np.trace((H-y).T.dot(H-y))

In [4]:
H =np.arange(dimension**2).reshape((dimension,dimension)) #This is just initial value of H
P =H 
B =np.zeros(shape=(dimension,dimension))
r=2

for i in range(iter):
    H =  minimize(fun=objective,x0=H.flatten(),method='CG',args=(P,B)).x.reshape(dimension,dimension)
    y_k = H+B
    u,d,v = np.linalg.svd(y_k,full_matrices=True)
    P=u.dot(np.eye(N=u.shape[1],M=v.shape[0])).dot(v)
    if not np.allclose(np.dot(P.T,P), np.identity(n=P.T.shape[0])):
        print "Constrained subproblem failed"
        break

    B = B+H-P

print "Is orthogonal constraint statisifed? Is printed matrix equal to I?",'\n'
print ((H.T.dot(H).astype(int)))

Is orthogonal constraint statisifed? Is printed matrix equal to I? 

[[  1076827001666225  20419611352248708  12023368862915824]
 [ 20419611352248708 548014470658839168 316070317908737600]
 [ 12023368862915824 316070317908737600 182496358854426368]]


In [27]:
(r*0.5/(2+r))

0.25